## Faiss Paper


- **Title**: _Billion scale similarity search with GPUs_
- 8.5x faster than previous GPU sota
- k-NN graph construction is very expensive to build
- Focuses on Product-Quantization (PQ) codes
- Similarity Search in vector collections:

$$
    L = k-argmin_{i=o:l} \Vert x-y_{i} \Vert_{2}
$$
- Approximate Nearest Neighbor search

### Flat

In [ ]:
import numpy as np
d = 764                           # dimension
nb = 1000000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible

In [ ]:
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [ ]:
xb.shape

In [ ]:
xq.shape

In [ ]:
xb[0]

In [ ]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

In [ ]:
index.ntotal

In [ ]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

In [ ]:
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

### IVFFlat

In [ ]:
import numpy as np

d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [ ]:
xb.shape

In [ ]:
xq.shape

In [ ]:
import faiss

nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [ ]:
assert not index.is_trained

In [ ]:
index.train(xb)
assert index.is_trained

In [ ]:
index.add(xb)                  # add may be a bit slower as well
D, I = index.search(xq, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries
index.nprobe = 10              # default nprobe is 1, try a few more
D, I = index.search(xq, k)
print(I[-5:])                  # neighbors of the 5 last queries
print(D[-5:])

### IVFPQ

In [1]:
import numpy as np

d = 256                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [4]:
import faiss

nlist = 100
m = 8
k = 5
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8) # 8 specifies that each sub-vector is encoded as 8 bits

In [5]:
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
index.nprobe = 10              # make comparable with experiment above
D, I = index.search(xq, k)     # search
print(I[-5:])

[[   0 1297  280  991   69]
 [   1  198  490  535  130]
 [   2  335  384  781  388]
 [   3  676   11  151  564]
 [   4  908 1168  870  110]]
[[16.379608 22.267044 23.142925 23.37597  23.527748]
 [17.832582 23.692463 25.317284 25.410645 25.428934]
 [17.849506 23.984257 24.000374 24.645636 24.664366]
 [16.672308 23.469244 23.581505 23.741306 23.874958]
 [15.994578 22.744059 22.813297 22.911263 23.175169]]
[[10901  9823 10297 10003  9370]
 [ 9462 10098  8798  9632  9526]
 [ 9552 10974  9965 10223  9500]
 [ 9772 11572  9949  9718  9954]
 [ 9272  9325 10114 10188 10228]]


In [6]:
D[-5:]

array([[21.794691, 22.022364, 22.046251, 22.23244 , 22.468832],
       [22.156908, 22.775482, 22.796028, 23.161697, 23.385788],
       [21.430569, 21.595377, 21.851482, 21.899704, 22.016834],
       [20.965322, 21.888481, 22.060913, 22.265953, 22.359808],
       [23.12369 , 23.25796 , 23.430618, 23.640244, 23.77743 ]],
      dtype=float32)

## Getting Started

[Getting Started](https://github.com/facebookresearch/faiss/wiki/Getting-started)

## References
1. [Video 1](https://www.youtube.com/watch?v=Un1Q92lfhPM)
2. [Product quantization for nearest neighbor search](https://lear.inrialpes.fr/pubs/2011/JDS11/jegou_searching_with_quantization.pdf)  
3. [Blog Post - A Simple Guide to Faiss](https://towardsdatascience.com/facebook-ai-similarity-search-7c564daee9eb)